In [35]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special
from scipy.integrate import quad
from scipy.optimize import curve_fit
from astropy import units as u
from astropy import constants as const
import time

# 1) Inelastic pp-interaction cross section $\sigma_{pp}$($E$)


In [36]:
def cross_section(E):
    L = np.log(E/10**3)
    E_th = 1.22
    return (34.3+1.88*L+0.25*pow(L,2))*pow((1-pow((E_th/E),4)),2)

# 2) CR energy luminosity due to supernovae 

In [37]:
def E_CR(RSN): #RSN in yr^-1
    E_SN_erg = 1e51*u.erg #characteristic energy output
    E_SN_GeV = (E_SN_erg.to(u.eV)).value*1e-9 # GeV
    xi = 0.10 # efficiency ~10%
    R_sn_yr = RSN*pow(u.yr,-1) # SN rate in yrs
    R_sn_s = (R_sn_yr.to(pow(u.second,-1))).value
    return R_sn_s*E_SN_GeV*xi


# 3) Normalisation constant $N_{C,SN}$

In [38]:
def integrand(p, alpha,pmax):
    m_p = 0.938 #GeV/c²
    p_max = pmax #GeV/c
    return (4*np.pi)*pow(p,2)*pow(p/m_p,-alpha)*np.exp(-p/p_max)*(np.sqrt(pow(p,2)+pow(0.938,2))-0.938) 


In [39]:

def N_C(ECR, alpha, pmax): #normalisation constant
    integral = integrate.quad(integrand,.0001,np.inf, args = (alpha,pmax))
    return ECR/integral[0]

# 4) Proton injection rate Q$_p$


In [40]:
def Q2(p,Rsn, R , al, N,pmax,h):
    R_pc = R*u.parsec  #pc to cm conversion
    R_cm = (R_pc.to(u.cm)).value
    h_pc = h*u.pc
    h_cm = (h_pc.to(u.cm)).value
    if h == 0: 
        V_spherical = (4/3)*np.pi*pow(R_cm,3)
        V_SBN = V_spherical
    else:
        V_disk = 2*h_cm*np.pi*pow(R_cm,2)
        V_SBN = V_disk
    m_p = 0.938
    p_max = pmax
    return (N/V_SBN)*pow(p/m_p,-al)*np.exp(-p/p_max)


# 5) Time scales 

## 5.1) Inelastic scatterings $\tau_{loss}$


In [41]:
def loss_time(E,nism,z): 
    eta = 0.51
    n_cm = nism*pow(u.cm,-3)
    n_m = (n_cm.to(pow(u.m,-3))).value
    
    if(z == 0):#u==0 then units are  year, otherwise seconds
        c_yr = (const.c.to(u.m*pow(u.yr,-1))).value
        sigma = cross_section(E)*1e-31 #mb -> m² 
        return 1/(eta*n_m*sigma*c_yr)
        
    else: 
        sigma = cross_section(E)*1e-31
        return 1/(eta*n_m*sigma*const.c.value)



## 5.2) Wind speed $\tau_{gwind}$

In [42]:
def tau_wind(R,v,z,h):
    
    if(z==0): #time scale in years
        if h==0:
            R_pc = R*u.parsec 
            R_SBN = (R_pc.to(u.m)).value
            v_wind_s = v*1000*pow(u.second,-1)
            v_wind_yr = (v_wind_s.to(pow(u.yr,-1))).value
            return (R_SBN/v_wind_yr) 
        else: 
            h_pc = h*u.parsec
            h_m = (h_pc.to(u.m)).value
            v_wind_s = v*1000*pow(u.second,-1)
            v_wind_yr = (v_wind_s.to(pow(u.yr,-1))).value
            return (h_m/v_wind_yr) 
            
    
    else:#time scale in seconds
        if h==0:
            R_pc = R*u.parsec 
            R_SBN = (R_pc.to(u.m)).value
            v_wind = v*1000 
            return (R_SBN/v_wind) 
        else:
            h_pc = h*u.parsec
            h_m = (h_pc.to(u.m)).value
            v_wind = v*1000 
            return (h_m/v_wind) 
            



## 5.3) total lifetime $\tau_{life}$

In [43]:
def tau_lifetime(R,vwind,E , nism,h):
    return pow(pow(tau_wind(R,vwind,1,h),-1)+pow(loss_time(E,nism,1),-1),-1)

# 6) Proton momentum distribution function $f_p$

In [44]:
def f_p(E, nism,R, v, Rsn, alpha, pmax,h):
    return tau_lifetime(R,v, E, nism,h)*Q2(E,Rsn,R,alpha,N_C(E_CR(Rsn),alpha,pmax),pmax,h)

# 7) Pion injection rate


In [45]:
def q_pi(E,nism,R,v,Rsn,alpha, pmax,h):
    n = nism*1e6 
    c = 3e8    
    K_pi = 0.17 # kinetic energy transfer parent proton to pion 
    m_p = 0.938 # GeV
    return (c*n)*(1/K_pi)*(cross_section((E/K_pi)+0.938)*1e-31)*4*np.pi*pow((E/K_pi)+0.938,2)*tau_lifetime(R,v, (E/K_pi)+0.938, nism,h)*Q2(E/K_pi+0.938,Rsn,R,alpha,N_C(E_CR(Rsn),alpha,pmax),pmax,h)

# 8) PDFs pion-to-neutrino decay 


In [46]:
def g_mu(x):
    r = 0.573
    return (9*pow(x,2)-6*np.log(x)-4*pow(x,3)-5)*(3-2*r)/(9*pow(1-r,2))

def h_mu1(x):
    r = 0.573
    return (9*pow(r,2)-6*np.log(r)-4*pow(r,3)-5)*(3-2*r)/(9*pow(1-r,2))

def h_mu2(x):
    r = 0.573
    return ((9*(r+x)-4*(pow(r,2)+r*x+pow(x,2)))*(1+2*r)*(r-x))/(9*pow(r,2))



def ge(x):
    r = 0.573
    return 2*(((1-x)*(6*(1-x)**2+r*(5+5*x-4*x**2))+6*r*np.log(x)))/(3*(1-r)**2)


def h_e1(x):
    r = 0.573
    return (2/(3*(pow((1-r),2))))*((1-r)*(6-7*r+11*pow(r,2)-4*pow(r,3))+6*r*np.log(r))

def h_e2(x):
    r = 0.573
    return((2*(r-x))/(3*pow(r,2))) *(7*pow(r,2)-4*pow(r,3)+7*x*r-4*x*pow(r,2)-2*pow(x,2)-4*pow(x,2)*r)

In [47]:
def f_mu1(x):
    lambd = 1-0.573
    if (lambd-x>=0):
        return 1/lambd
    else: 
        return 0

def f_mu2(x):
    r = 0.573
    if (x-r>=0):
        return g_mu(x)
    if (r-x>=0):
        return h_mu1(x)+h_mu2(x)
        
    else:
        return 0

def f_ve(x):
    r =0.573
    if (x - r >= 0):
        return ge(x)
    if (r-x > 0):
        return h_e1(x)+h_e2(x)
    else:
        return 0

# 8) Neutrino production rate $q_{\nu}/3$


In [48]:
def q_nu(x,E_nu, nism,R,v,Rsn,alpha,pmax,h):
    return (2/3)*(f_mu1(x)+f_mu2(x)+f_ve(x))*q_pi(E_nu/x, nism,R,v,Rsn,alpha,pmax,h)*(1/x)

# 9) Flux generator 

In [49]:
def flux_generator_vol(nism,R,v,Rsn,alpha,pmax,D_L,h):
    DL_pc = (D_L*1e6)*u.parsec 
    DL_cm = (DL_pc.to(u.cm)).value
    R_pc = R*u.pc
    R_cm = (R_pc.to(u.cm)).value
    h_pc = h*u.parsec
    h_cm = (h_pc.to(u.cm)).value
    
    E = np.arange(1e2,1e5,1e3)
    E2 = np.arange(1e5,1e7,1e5)
    
    fluxes = []
    fluxes2 = []
    for i in E: 
        I = quad(q_nu,0,1,args = (i,nism,R,v,Rsn,alpha,pmax,h) )[0] #takes factor three already into account
        fluxes += [I]
    for i in E2: 
        I = quad(q_nu,0,1,args = (i,nism,R,v,Rsn,alpha,pmax,h) )[0] #takes factor three already into account
        fluxes2 += [I]
        
    fluxes = np.array(fluxes)
    fluxes2 = np.array(fluxes2)
   
    V = 2*h_cm*np.pi*pow(R_cm , 2 )
    
    scaled_flux = (V/(4*np.pi*pow(DL_cm,2)))*pow(E,2)*fluxes
    scaled_flux2 = (V/(4*np.pi*pow(DL_cm,2)))*pow(E2,2)*fluxes2
    scaled_flux_final = np.concatenate((scaled_flux,scaled_flux2))
    
    return [np.concatenate((E,E2)),scaled_flux_final]

In [50]:
def gas_surf_density(SFR_surface_density):
    return pow(10,4.6/1.9)*pow(SFR_surface_density, 1/1.9)

def Volume_density_LIRGs(Gas_dens,R,mu,ratio):
    H = R*1000*ratio # pc 
    M_sol = (u.solMass).to(u.kg) #kg
    mp = (const.m_p).value #kg
    nism_pc3 = (pow(2*H*(mp/M_sol)*mu,-1)*Gas_dens*pow(u.pc,-3)) #pc-3
    nism_cm3 = (nism_pc3.to(pow(u.cm,-3))).value #cm-3
    return nism_cm3

def IR(IR_surface,Re):
    return IR_surface*np.pi*pow(Re,2)

def SNr(LIR): 
    return 2.7e-12*LIR  


In [81]:
def flux_generator_vol2(gas_surf_density,R,v,IR_surface_brightness,D_L,alpha, pmax , frac , mu, name):
    
    nism = Volume_density_LIRGs(gas_surf_density,R, mu, frac)
    Rsn = SNr(IR(IR_surface_brightness,R))
    
    DL_pc = (D_L*1e6)*u.parsec 
    DL_cm = (DL_pc.to(u.cm)).value
    R_pc = (R*1000)*u.pc
    R_cm = (R_pc.to(u.cm)).value
    
    h_pc = (R*frac*1000)*u.parsec
    h_cm = (h_pc.to(u.cm)).value
    
    E = np.arange(1e2,1e5,1e3)
    E2 = np.arange(1e5,1e7,1e5)
    
    fluxes = []
    fluxes2 = []
    for i in E: 
        I = quad(q_nu,0,1,args = (i,nism,R*1000,v,Rsn,alpha,pmax,h_pc.value) )[0] #takes factor three already into account
        fluxes += [I]
    for i in E2: 
        I = quad(q_nu,0,1,args = (i,nism,R*1000,v,Rsn,alpha,pmax,h_pc.value) )[0] #takes factor three already into account
        fluxes2 += [I]
        
    fluxes = np.array(fluxes)
    fluxes2 = np.array(fluxes2)
   
    V = 2*h_cm*np.pi*pow(R_cm,2)
    
    scaled_flux = (V/(4*np.pi*pow(DL_cm,2)))*pow(E,2)*fluxes
    scaled_flux2 = (V/(4*np.pi*pow(DL_cm,2)))*pow(E2,2)*fluxes2
    scaled_flux_final = np.concatenate((scaled_flux,scaled_flux2))
    print('done')
    return [np.concatenate((E,E2)),scaled_flux_final,name]